In [4]:
from haystack.preprocessor.utils import convert_files_to_dicts

In [6]:
dicts=convert_files_to_dicts("./data/", split_paragraphs=True) # no cleaning function applied

12/09/2020 10:56:21 - INFO - haystack.preprocessor.utils -   Converting data/9781839217579-THE_DEEP_LEARNING_WITH_KERAS_WORKSHOP_SECOND_EDITION.pdf


In [7]:
type(dicts), len(dicts), type(dicts[0]), dicts[0].keys()

(list, 1418, dict, dict_keys(['text', 'meta']))

In [8]:
print(dicts[0])

{'text': '\x0cThe Deep Learning\nwith Keras Workshop\nSecond Edition', 'meta': {'name': '9781839217579-THE_DEEP_LEARNING_WITH_KERAS_WORKSHOP_SECOND_EDITION.pdf'}}


In [9]:
from haystack.document_store.faiss import FAISSDocumentStore

document_store = FAISSDocumentStore()
document_store.delete_all_documents()

In [10]:
document_store.write_documents(dicts)

In [11]:
from haystack.retriever.dense import DensePassageRetriever
retriever = DensePassageRetriever(document_store=document_store,
                                  query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
                                  passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
                                  max_seq_len_query=64,
                                  max_seq_len_passage=256,
                                  batch_size=16,
                                  use_gpu=False,
                                  embed_title=True,
                                  use_fast_tokenizers=True)
    

12/09/2020 10:56:39 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.
12/09/2020 10:56:46 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.


In [12]:
document_store.update_embeddings(retriever)

12/09/2020 10:56:58 - INFO - haystack.document_store.faiss -   Updating embeddings for 1418 docs...
Inferencing Samples: 100%|██████████| 89/89 [11:22<00:00,  7.67s/ Batches]
12/09/2020 11:08:23 - INFO - haystack.document_store.faiss -   Indexing embeddings and updating vectors_ids...
100%|██████████| 1/1 [00:00<00:00,  7.54it/s]


In [13]:
from haystack.reader.farm import FARMReader
farm_reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

12/09/2020 11:08:31 - INFO - farm.utils -   device: cpu n_gpu: 0, distributed training: False, automatic mixed precision training: None
12/09/2020 11:08:31 - INFO - farm.infer -   Could not find `deepset/roberta-base-squad2` locally. Try to download from model hub ...
Some weights of RobertaModel were not initialized from the model checkpoint at deepset/roberta-base-squad2 and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
12/09/2020 11:08:36 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.
12/09/2020 11:08:45 - INFO - farm.utils -   device: cpu n_gpu: 0, distributed training: False, automatic mixed precision training: None
12/09/2020 11:08:45 - INFO - farm.inf

In [14]:
from haystack import Finder
finder = Finder(farm_reader, retriever)

In [15]:
prediction = finder.get_answers(question="What is RNN?", top_k_retriever=10, top_k_reader=3)

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  4.99 Batches/s]
12/09/2020 11:08:51 - INFO - haystack.finder -   Got 10 candidates from retriever
12/09/2020 11:08:51 - INFO - haystack.finder -   Reader is looking for detailed answer in 3458 chars ...
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  1.83 Batches/s]


In [16]:
from haystack.utils import print_answers

In [17]:
print_answers(prediction, details="medium")

[   {   'answer': 'the hidden layer not only gives the\n'
                  'output, but it also feeds back the information of the '
                  'output into itself',
        'context': 'y of the RNN is that the hidden layer not only gives the\n'
                   'output, but it also feeds back the information of the '
                   'output into itself. Before taking a\n'
                   'dee',
        'score': 12.397671699523926},
    {   'answer': 'Recurrent Neural Networks',
        'context': 'Recurrent Neural Networks (RNNs)\n'
                   'RNNs are a class of neural networks that are built on the '
                   'concept of sequential\n'
                   'memory. Unlike traditional neural net',
        'score': 11.025992393493652},
    {   'answer': 'Long Short-Term Memory',
        'context': 'Long Short-Term Memory (LSTM)\n'
                   'LSTMs are RNNs whose main objective is to overcome the '
                   'shortcomings of the vanishin

In [18]:
prediction = finder.get_answers(question="What is a layer?", top_k_retriever=10, top_k_reader=3)

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  3.89 Batches/s]
12/09/2020 11:09:04 - INFO - haystack.finder -   Got 10 candidates from retriever
12/09/2020 11:09:04 - INFO - haystack.finder -   Reader is looking for detailed answer in 5180 chars ...
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  1.97 Batches/s]


In [19]:
print_answers(prediction, details="medium")

[   {   'answer': 'Dense layer',
        'context': 'ras. For now, we will use only the simplest type of\n'
                   'layer, called the Dense layer. A Dense layer is equivalent '
                   'to the fully connected\n'
                   'layers that we h',
        'score': 10.62425708770752},
    {   'answer': 'a\ncomposition of nodes',
        'context': 'ers is part of the Keras core API. A layer can be thought '
                   'of as a\n'
                   'composition of nodes, and at each node, a set of '
                   'computations happen. In Keras, all ',
        'score': 10.560615539550781},
    {   'answer': 'convolutional',
        'context': 'd: it has a four-dimensional input shape (None, 224, 224,\n'
                   '3) and it has three convolutional layers.\n'
                   'The last four layers of the output are as follows:',
        'score': 9.095014572143555}]


In [20]:
prediction = finder.get_answers(question="What is deep learning?", top_k_retriever=10, top_k_reader=3)

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  4.34 Batches/s]
12/09/2020 11:09:17 - INFO - haystack.finder -   Got 10 candidates from retriever
12/09/2020 11:09:17 - INFO - haystack.finder -   Reader is looking for detailed answer in 3292 chars ...
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  2.14 Batches/s]


In [21]:
print_answers(prediction, details="medium")

[   {   'answer': 'Keras',
        'context': "oss')\n"
                   "plt.xlabel('epoch')\n"
                   "plt.legend(['train loss', 'validation loss'], loc='upper "
                   "right')\n"
                   '\x0c'
                   'Chapter 3: Deep Learning with Keras | 337\n'
                   'Expected output:',
        'score': 6.224386215209961},
    {   'answer': 'cyclical: 284',
        'context': ', 254, 261, 263,\n'
                   '266, 269, 271, 274,\n'
                   '276, 294, 298, 301,\n'
                   '305-306, 308-309\n'
                   'current: 95, 113, 283,\n'
                   '287, 294, 301\n'
                   'curves: 193-194, 204, 222\n'
                   'cyclical: 284',
        'score': -4.464661598205566},
    {   'answer': 'gradient',
        'context': '237, 248, 250, 259,\n'
                   '268, 292, 298, 304\n'
                   'google: 258, 277,\n'
                   '282-283, 286, 308\n'
                   '

In [22]:
prediction = finder.get_answers(question="What is logistic regression?", top_k_retriever=10, top_k_reader=3)


Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  4.78 Batches/s]
12/09/2020 11:09:30 - INFO - haystack.finder -   Got 10 candidates from retriever
12/09/2020 11:09:30 - INFO - haystack.finder -   Reader is looking for detailed answer in 9211 chars ...
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  2.40 Batches/s]


In [23]:
print_answers(prediction, details="medium")

[   {   'answer': 'feature\n'
                  'coefficients are learned and predictions are made by taking '
                  'the sum of the product\n'
                  'of the feature coefficients and features',
        'context': ' in which feature\n'
                   'coefficients are learned and predictions are made by '
                   'taking the sum of the product\n'
                   'of the feature coefficients and features.\n'
                   '• Decis',
        'score': 10.148681640625},
    {   'answer': 'a very simple neural network with only one hidden layer and '
                  'only\n'
                  'one node in its hidden layer',
        'context': 'logistic\n'
                   'regression involves a very simple neural network with only '
                   'one hidden layer and only\n'
                   'one node in its hidden layer.\n'
                   'An overview of the logisti',
        'score': 10.00578498840332},
    {   'answer': 'ridge

In [24]:
prediction = finder.get_answers(question="What is early stopping?", top_k_retriever=10, top_k_reader=3)

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  5.10 Batches/s]
12/09/2020 11:09:41 - INFO - haystack.finder -   Got 10 candidates from retriever
12/09/2020 11:09:41 - INFO - haystack.finder -   Reader is looking for detailed answer in 6129 chars ...
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  2.24 Batches/s]


In [25]:
print_answers(prediction, details="medium")

[   {   'answer': 'forcing the Keras model to stop the training when a desired '
                  'metric—for example,\n'
                  'the test error rate—is not improving anymore',
        'context': '. This\n'
                   'means forcing the Keras model to stop the training when a '
                   'desired metric—for example,\n'
                   'the test error rate—is not improving anymore. In order to',
        'score': 10.376293182373047},
    {   'answer': 'The training',
        'context': 'ovide it as a callbacks\n'
                   'argument to model.fit() and train the model. The training '
                   'will automatically stop\n'
                   'according to the EarlyStopping() callback:\n'
                   'h',
        'score': 6.640634536743164},
    {   'answer': 'the training\n'
                  'process automatically stops after about 150 epochs',
        'context': 'ing callback with patience=10 to the model, the training\n'
          

In [26]:
prediction = finder.get_answers(question="What is early stopping used for?", top_k_retriever=10, top_k_reader=3)

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  5.10 Batches/s]
12/09/2020 11:09:52 - INFO - haystack.finder -   Got 10 candidates from retriever
12/09/2020 11:09:52 - INFO - haystack.finder -   Reader is looking for detailed answer in 6771 chars ...
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  1.07 Batches/s]


In [27]:
print_answers(prediction, details="medium")

[   {   'answer': 'forcing the Keras model to stop the training when a desired '
                  'metric—for example,\n'
                  'the test error rate—is not improving anymore',
        'context': '. This\n'
                   'means forcing the Keras model to stop the training when a '
                   'desired metric—for example,\n'
                   'the test error rate—is not improving anymore. In order to',
        'score': 8.451149940490723},
    {   'answer': 'to prevent your Keras model from\n'
                  'overfitting the training data. To do this, you utilized the '
                  'EarlyStopping callback\n'
                  'and trained the model with it. We used this callback to '
                  'stop the model any time the\n'
                  'validation loss increased',
        'context': ' to prevent your Keras model from\n'
                   'overfitting the training data. To do this, you utilized '
                   'the EarlyStopping callb

In [28]:
# EVALUATING

In [29]:
doc_index = "evaluation_docs"
label_index = "evaluation_labels"

filename="./data/nq/nq_dev_subset_v2.json"

In [31]:
from haystack.preprocessor.utils import eval_data_from_file
docs, labels = eval_data_from_file(filename=filename) # return: (List of Documents, List of Labels)

In [32]:
from haystack.document_store.faiss import FAISSDocumentStore
document_store = FAISSDocumentStore()

In [33]:
document_store.write_documents(docs, index=doc_index)

In [34]:
from haystack.retriever.dense import DensePassageRetriever
dpr = DensePassageRetriever(document_store=document_store,
                                  query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
                                  passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
                                  max_seq_len_query=64,
                                  max_seq_len_passage=256,
                                  batch_size=16,
                                  use_gpu=False,
                                  embed_title=True,
                                  use_fast_tokenizers=True)

12/09/2020 11:11:07 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.
12/09/2020 11:11:14 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.


In [35]:
document_store.update_embeddings(dpr, index=doc_index)

12/09/2020 11:11:17 - INFO - haystack.document_store.faiss -   Updating embeddings for 50 docs...
Inferencing Samples: 100%|██████████| 4/4 [00:21<00:00,  5.42s/ Batches]
12/09/2020 11:11:40 - INFO - haystack.document_store.faiss -   Indexing embeddings and updating vectors_ids...
100%|██████████| 1/1 [00:00<00:00, 93.36it/s]


In [36]:
document_store.write_labels(labels, index=label_index)

In [40]:
## Evaluate Retriever on its own
dpr_eval_results = dpr.eval(top_k=5, label_index=label_index, doc_index=doc_index)

12/09/2020 11:12:29 - INFO - haystack.retriever.base -   Performing eval queries...
100%|██████████| 54/54 [00:11<00:00,  4.59it/s]
12/09/2020 11:12:41 - INFO - haystack.retriever.base -   For 52 out of 54 questions (96.30%), the answer was in the top-5 candidate passages selected by the retriever.


In [41]:
## Evaluate Retriever on its own
dpr_eval_results = dpr.eval(top_k=3, label_index=label_index, doc_index=doc_index)

12/09/2020 11:13:12 - INFO - haystack.retriever.base -   Performing eval queries...
100%|██████████| 54/54 [00:11<00:00,  4.78it/s]
12/09/2020 11:13:23 - INFO - haystack.retriever.base -   For 52 out of 54 questions (96.30%), the answer was in the top-3 candidate passages selected by the retriever.


In [42]:
## Retriever Recall is the proportion of questions for which the correct document containing the answer is
## among the correct documents
print("Retriever Recall:", dpr_eval_results["recall"])
## Retriever Mean Avg Precision rewards retrievers that give relevant documents a higher rank
print("Retriever Mean Avg Precision:", dpr_eval_results["map"])

Retriever Recall: 0.9629629629629629
Retriever Mean Avg Precision: 0.9537037037037037


In [43]:
from haystack.reader.farm import FARMReader
farm_reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=False)

12/09/2020 11:13:37 - INFO - farm.utils -   device: cpu n_gpu: 0, distributed training: False, automatic mixed precision training: None
12/09/2020 11:13:37 - INFO - farm.infer -   Could not find `deepset/roberta-base-squad2` locally. Try to download from model hub ...
Some weights of RobertaModel were not initialized from the model checkpoint at deepset/roberta-base-squad2 and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
12/09/2020 11:13:42 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.
12/09/2020 11:13:51 - INFO - farm.utils -   device: cpu n_gpu: 0, distributed training: False, automatic mixed precision training: None
12/09/2020 11:13:51 - INFO - farm.inf

In [44]:
from farm.utils import initialize_device_settings

device, n_gpu = initialize_device_settings(use_cuda=False)

12/09/2020 11:17:26 - INFO - farm.utils -   device: cpu n_gpu: 0, distributed training: False, automatic mixed precision training: None


In [45]:
# Evaluate Reader on its own
reader_eval_results = farm_reader.eval(document_store=document_store, device=device, label_index=label_index, doc_index=doc_index)
# Evaluation of Reader can also be done directly on a SQuAD-formatted file without passing the data to Elasticsearch
#reader_eval_results = reader.eval_on_file("../data/nq", "nq_dev_subset_v2.json", device=device)

## Reader Top-N-Accuracy is the proportion of predicted answers that match with their corresponding correct answer
print("Reader Top-N-Accuracy:", reader_eval_results["top_n_accuracy"])
## Reader Exact Match is the proportion of questions where the predicted answer is exactly the same as the correct answer
print("Reader Exact Match:", reader_eval_results["EM"])
## Reader F1-Score is the average overlap between the predicted answers and the correct answers
print("Reader F1-Score:", reader_eval_results["f1"])

12/09/2020 11:17:35 - INFO - haystack.reader.farm -   Performing Evaluation using top_k_per_candidate = 3 
and consequently, QuestionAnsweringPredictionHead.n_best = 4. 
This deviates from FARM's default where QuestionAnsweringPredictionHead.n_best = 5
Evaluating: 100%|██████████| 73/73 [27:45<00:00, 22.82s/it]Reader Top-N-Accuracy: 0.6111111111111112
Reader Exact Match: 0.2777777777777778
Reader F1-Score: 0.30750487329434695

